# Sygus completions

In [ ]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

In [ ]:
from sygus.utils import sample_gpt_solutions, SygusProblem, SygusBenchmark, cleanup_completion, add_sygus_prefix

In [ ]:
import typing as t
from pprint import pprint
from dataclasses import dataclass
import sexpdata as sexp
from sexpdata import Symbol
import json
import random
import math
import re

In [ ]:
BENCHMARKS_DIRECTORY = ROOT_DIRECTORY / "sygus/Probe/src/test/benchmarks"
CIRCUIT_DIRECTORY = BENCHMARKS_DIRECTORY / "circuit/test"
HACKERS_DELIGHT_DIRECTORY = BENCHMARKS_DIRECTORY / "hackers-delight"
LARGER_STRING_GRAMMAR_DIRECTORY = BENCHMARKS_DIRECTORY / "larger-grammar"
STRING_GRAMMAR_DIRECTORY = BENCHMARKS_DIRECTORY / "string"

## testing completion cleanup and parsing

In [ ]:
EXAMPLES = [
    '```python\n(define-fun f ((_arg_0 String)) String (str.substr _arg_0 (+ (str.indexof _arg_0 ".") 1) (str.len _arg_0)))\n```',
    '\n  (str.substr _arg_0 (+ (str.indexof _arg_0 ".") 1) (str.len _arg_0))\n)',
    '`(str.substr _arg_0 (+ (str.indexof _arg_0 ".") 1) (str.len _arg_0))`',
    '(define-fun f ((_arg_0 String)) String (str.++ (str.++ \"R/\" (int.to.str (str.to.int (str.substr _arg_0 5 3)))) \" \" (int.to.str (str.to.int (str.substr _arg_0 9 3))) \" \" (int.to.str (str.to.int (str.substr _arg_0 13 3)))))',
    ") (str.replace _arg_0 \",\" \"\") \",\")",
    "```python\n(str.substr name 6 3)\n```"
]

for example in EXAMPLES:
    print("original")
    print(example)
    print("cleaned")
    print(cleanup_completion(example))
    print()

In [ ]:
EXAMPLES = [
    ('(define-fun f ((_arg_0 String)) String (str.++ (str.++ \"R/\" (int.to.str (str.to.int (str.substr _arg_0 5 3)))) \" \" (int.to.str (str.to.int (str.substr _arg_0 9 3))) \" \" (int.to.str (str.to.int (str.substr _arg_0 13 3)))))', '(define-fun f (_arg_0 String) String'),
    ("(str.replace _arg_0 \",\" \"\") \",\")", '(define-fun f (_arg_0 String) String')
]

for completion, prefix in EXAMPLES:
    print("original")
    print(completion)
    print("with prefix")
    print(add_sygus_prefix(completion, prefix))
    print()


## Generating Completions

### Larger String Grammar

In [ ]:
LARGER_STRING_GRAMMAR_OUTPUT_FILE = ROOT_DIRECTORY / "sygus/larger-string-grammar-completions.gpt-3.5-turbo.json"

In [ ]:
if LARGER_STRING_GRAMMAR_OUTPUT_FILE.exists():
    LARGER_STRING_BENCHMARK = SygusBenchmark.read_from_file(LARGER_STRING_GRAMMAR_OUTPUT_FILE, LARGER_STRING_GRAMMAR_DIRECTORY)
else:
    LARGER_STRING_BENCHMARK = SygusBenchmark(
        LARGER_STRING_GRAMMAR_DIRECTORY
    )

In [ ]:
print(LARGER_STRING_BENCHMARK.sygus["exceljet2modified.sl"].user_message)

In [ ]:
for filename, problem in LARGER_STRING_BENCHMARK.sygus.items():
    pprint(filename)
    pprint(problem.signature)
    pprint(problem.examples)

In [ ]:
LARGER_STRING_BENCHMARK.sample_solutions(model="gpt-3.5-turbo", n=20)

In [ ]:
LARGER_STRING_BENCHMARK.write(LARGER_STRING_GRAMMAR_OUTPUT_FILE)

### String Grammar

In [ ]:
STRING_BENCHMARK_OUTPUT_FILE = ROOT_DIRECTORY / "sygus/string-grammar-completions.gpt-3.5-turbo.json"

In [ ]:
if STRING_BENCHMARK_OUTPUT_FILE.exists():
    STRING_BENCHMARK = SygusBenchmark.read_from_file(STRING_BENCHMARK_OUTPUT_FILE, STRING_GRAMMAR_DIRECTORY)
else:
    STRING_BENCHMARK = SygusBenchmark(
        STRING_GRAMMAR_DIRECTORY
    )

In [ ]:
print(STRING_BENCHMARK.sygus["get-first-name-from-name-with-comma.sl"].user_message)

In [ ]:
for filename, problem in STRING_BENCHMARK.sygus.items():
    pprint(filename)
    pprint(problem.signature)
    pprint(problem.examples)
    pprint(len(STRING_BENCHMARK.output.get(filename, {
        "completions": []
    })["completions"]))

In [ ]:
STRING_BENCHMARK.sample_solutions(model="gpt-3.5-turbo", n=20)

In [ ]:

STRING_BENCHMARK.write(STRING_BENCHMARK_OUTPUT_FILE)

### Circuit

In [ ]:
CIRCUIT_OUTPUT_FILE = ROOT_DIRECTORY / "sygus/circuit-completions.gpt-3.5-turbo.json"

In [ ]:
CIRCUIT_EXAMPLES_FILE = ROOT_DIRECTORY / "sygus/io-results-circuit.json"
CIRCUIT_EXAMPLES_JSON = json.loads(CIRCUIT_EXAMPLES_FILE.read_text())
pprint(CIRCUIT_EXAMPLES_JSON["CrCy_10-sbox2-D5-sIn79.sl"])

CIRCUIT_EXAMPLES = {}

for filename, examples in CIRCUIT_EXAMPLES_JSON.items():
    example_tuples = [(example["inputs"], example["output"]) for example in examples]
    CIRCUIT_EXAMPLES[filename] = random.sample(example_tuples, 10) if len(example_tuples) > 10 else example_tuples

pprint(CIRCUIT_EXAMPLES["CrCy_10-sbox2-D5-sIn79.sl"])

In [ ]:
if CIRCUIT_OUTPUT_FILE.exists():
    CIRCUIT_BENCHMARK = SygusBenchmark.read_from_file(CIRCUIT_OUTPUT_FILE, CIRCUIT_DIRECTORY, CIRCUIT_EXAMPLES)
else:
    CIRCUIT_BENCHMARK = SygusBenchmark(
        CIRCUIT_DIRECTORY, CIRCUIT_EXAMPLES
    )

In [ ]:
print(CIRCUIT_BENCHMARK.sygus["CrCy_10-sbox2-D5-sIn79.sl"].user_message)

In [ ]:
for filename, problem in CIRCUIT_BENCHMARK.sygus.items():
    pprint(filename)
    pprint(problem.signature)
    pprint(problem.examples)

In [ ]:
CIRCUIT_BENCHMARK.sample_solutions(model="gpt-3.5-turbo", n=20)

In [ ]:
CIRCUIT_BENCHMARK.write(CIRCUIT_OUTPUT_FILE)

### Hacker's Delight

In [ ]:
HACKERS_DELIGHT_OUTPUT_FILE = ROOT_DIRECTORY / "sygus/hackers-delight-completions.gpt-3.5-turbo.json"

In [ ]:
HACKERS_DELIGHT_EXAMPLE_FILE = ROOT_DIRECTORY / "sygus/io-results-bitvec.json"
HACKERS_DELIGHT_EXAMPLES_JSON = json.loads(HACKERS_DELIGHT_EXAMPLE_FILE.read_text())
pprint(HACKERS_DELIGHT_EXAMPLES_JSON["hd-08.sl"])

HACKERS_DELIGHT_EXAMPLES = {}

for filename, examples in HACKERS_DELIGHT_EXAMPLES_JSON.items():
    example_tuples = [
        (example["inputs"], example["output"]) for example in examples
    ]
    HACKERS_DELIGHT_EXAMPLES[filename] = random.sample(example_tuples, 10) if len(example_tuples) > 10 else example_tuples

pprint(HACKERS_DELIGHT_EXAMPLES["hd-08.sl"])

In [ ]:
if HACKERS_DELIGHT_OUTPUT_FILE.exists():
    HACKERS_DELIGHT_BENCHMARK = SygusBenchmark.read_from_file(HACKERS_DELIGHT_OUTPUT_FILE, HACKERS_DELIGHT_DIRECTORY, HACKERS_DELIGHT_EXAMPLES)
else:
    HACKERS_DELIGHT_BENCHMARK = SygusBenchmark(
        HACKERS_DELIGHT_DIRECTORY, HACKERS_DELIGHT_EXAMPLES
    )

In [ ]:
print(HACKERS_DELIGHT_BENCHMARK.sygus["hd-08.sl"].user_message)

In [ ]:
for filename, problem in HACKERS_DELIGHT_BENCHMARK.sygus.items():
    pprint(filename)
    pprint(problem.signature)
    pprint(problem.examples)

In [ ]:
HACKERS_DELIGHT_BENCHMARK.sample_solutions(model="gpt-3.5-turbo", n=20)

In [ ]:
HACKERS_DELIGHT_BENCHMARK.write(HACKERS_DELIGHT_OUTPUT_FILE)